# Convert Model to ONNX

In [ ]:
%cd PFD

In [3]:
%matplotlib inline
import cv2, numpy as np, matplotlib.pyplot as plt
import torch
import torchvision
from torchvision.models import ResNet50_Weights
from torchvision.models.detection import KeypointRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F
import time
import tensorflow as tf
import torchvision.transforms as T
import onnx
import onnxruntime as onnxrt
from onnx_tf.backend import prepare
from onnxruntime.quantization import quantize_dynamic, quantize_static, QuantType, QuantFormat

%matplotlib inline

In [ ]:
def get_model(num_keypoints, weights_path=None):
    
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 2.0, 3.0, 4.0))
    model = torchvision.models.detection.keypointrcnn_resnet50_fpn(weights=None,
                                                                   weights_backbone=ResNet50_Weights.DEFAULT,
                                                                   num_keypoints=num_keypoints,
                                                                   num_classes = 2, # Background is the first class, object is the second class
                                                                   rpn_anchor_generator=anchor_generator, 
                                                                   min_size=512)

    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model = get_model(num_keypoints = 4, weights_path='./keypoint_model/weights/keypointsrcnn_weights.pth')
model.to(device)
print('done')

## Export Model

In [ ]:
# init random data to convert model
frame_np_1 = np.random.rand(3, 512, 512)
frame_np_1 = torch.FloatTensor(frame_np_1)
frame_np_1.to(device)

frame_np_2 = np.random.rand(3, 512, 512)
frame_np_2 = torch.FloatTensor(frame_np_1)
frame_np_2.to(device)

x_reshaped = [frame_np_1, frame_np_2]

In [ ]:
# move model to cpu
model.to('cpu')

# convert pytorch model to onnx
torch.onnx.export(
    model, 
    [frame_np_1, frame_np_2], 
    "./assets/keypoint_rcnn_min.onnx", 
    input_names=['input_1'],
    output_names=['boxes_1', 'labels_1', 'scores_1', 'keypoints_1', 'keypoints_scores_1'],
    dynamic_axes={'input_1': {1: 'height', 2: 'width'}},
    opset_version = 11
)

In [ ]:
# quantize model for mobile use
quantize_dynamic(
    './assets/keypoint_rcnn_min.onnx', 
    './assets/keypoint_rcnn_min_quant.onnx',
    weight_type=QuantType.QUInt8, 
)